<a href="https://colab.research.google.com/github/ukab23/Multiclass-Text-Classification/blob/master/ULMfit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.2MB 22kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x635e6000 @  0x7f5d27aa52a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [2]:
!pip install fastai

    100% |████████████████████████████████| 143kB 8.2MB/s 
    100% |████████████████████████████████| 61kB 21.8MB/s 
    100% |████████████████████████████████| 163kB 13.1MB/s 
    100% |████████████████████████████████| 112kB 30.2MB/s 
    100% |████████████████████████████████| 1.9MB 8.8MB/s 
    100% |████████████████████████████████| 23.3MB 1.5MB/s 
  Running setup.py bdist_wheel for bottleneck ... - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/f2/bf/ec/e0f39aa27001525ad455139ee57ec7d0776fe074dfd78c97e4
Successfully built bottleneck
spacy 2.0.16 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
torchvision-nightly 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: thinc 6.12.1
    Uninstalling thinc-6.12.1:
      Successfully uninstalled thinc-6.12.1
  Found existing installation: spacy 2.0.18
    Uninstalling spacy-2.0.18:
      Successful

In [0]:
import fastai
from fastai import *
from fastai.text import *
import pandas as pd
import numpy as np
import io
import os
from functools import partial

In [8]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data                           

In [12]:
df = pd.DataFrame({'Label':dataset.target,'Text':dataset.data})
df.head(10)

,Label,Text
0,17,Well i'm not sure about the story nad it did s...
1,0,"\n\n\n\n\n\n\nYeah, do you expect people to re..."
2,17,Although I realize that principle is not one o...
3,11,Notwithstanding all the legitimate fuss about ...
4,10,"Well, I will have to change the scoring on my ..."
5,15,"\n \nI read somewhere, I think in Morton Smit..."
6,4,\nOk. I have a record that shows a IIsi with ...
7,17,\n\n\nSounds like wishful guessing.\n\n\n\n\n'...
8,13,Nobody is saying that you shouldn't be allowe...
9,12,\n I was wondering if anyone can shed any lig...


In [0]:
df = df[df['Label'].isin([1,10])]

In [17]:
df['Label'].value_counts()

10    600
1     584
Name: Label, dtype: int64

In [0]:
df = df.reset_index(drop = True)

In [0]:
df['Text'] = df['Text'].str.replace("[^a-zA-Z]"," ")

In [21]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [0]:
tokenized_doc = df['Text'].apply(lambda x:x.split())

In [0]:
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [0]:
detokenized_doc = []
for i in range(len(df)):
  t = " ".join(tokenized_doc[i])
  detokenized_doc.append(t)
df['Text'] = detokenized_doc  

In [0]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df, stratify = df['Label'], random_state = 12, test_size = 0.4)

In [28]:
df_train.shape

(710, 2)

In [29]:
df_val.shape

(474, 2)

In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_train, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_train, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.7)

In [33]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,7.783910,6.285647,0.140609


In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, drop_mult=0.7)

In [0]:
learn.load_encoder('ft_enc')

In [37]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.500247,0.351566,0.886076


In [0]:
preds, target = learn.get_preds()

In [45]:
predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, target)

col_0,0,1
row_0,,
0,213,33
1,21,207


In [51]:
for i in range(10):
  print(preds[i])

tensor([0.9963, 0.0037])
tensor([0.0112, 0.9888])
tensor([0.5617, 0.4383])
tensor([0.1056, 0.8944])
tensor([0.3443, 0.6557])
tensor([0.1174, 0.8826])
tensor([0.0190, 0.9810])
tensor([0.0411, 0.9589])
tensor([0.0584, 0.9416])
tensor([0.1145, 0.8855])


In [52]:
for j in range(10):
  print(target[j])

tensor(0)
tensor(1)
tensor(1)
tensor(1)
tensor(0)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
